In [1]:
pip install transformers datasets sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import Dataset
from transformers import AutoTokenizer
import re

# Load the Arabic text file
with open("/content/cairo_tourism_guide_arabic_only.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# Split into chunks by landmarks or locations
chunks = re.split(r"\n\s*\n", raw_text)
chunks = [chunk.strip() for chunk in chunks if chunk.strip()]

# Create a Dataset object
data = Dataset.from_dict({"text": chunks})


In [3]:
data_df = data.to_pandas()
data_df.head()

,text
0,أهرامات الجيزة وتمثال أبو الهول
1,الموقع \nتوجد الأهرامات الثلاثة وأبو الهول في ...
2,أهم الأنشطة في الأهرامات \nستجد العديد من الأن...
3,أوقات الزيارة \nيمكنك الدخول إلى الاهرامات الث...
4,رسوم دخول الاهرامات \nسعر تذكرة زيارة الأهراما...


In [5]:
from sentence_transformers import SentenceTransformer

# Load the correct public model
model = SentenceTransformer("intfloat/multilingual-e5-large")

# Encode your chunks
embeddings = model.encode(data["text"], show_progress_bar=True)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:

# Display the shape of the embeddings to confirm success
(len(embeddings), len(embeddings[0]))

(94, 1024)

In [7]:
import faiss
import numpy as np

# Convert embeddings to numpy array (if not already)
embedding_matrix = np.array(embeddings)

# Define the FAISS index (L2 or cosine similarity; we’ll use inner product with normalized vectors for cosine sim)
dimension = embedding_matrix.shape[1]
index = faiss.IndexFlatIP(dimension)  # IP = inner product

# Normalize the vectors (required for cosine similarity)
faiss.normalize_L2(embedding_matrix)

# Add vectors to index
index.add(embedding_matrix)

print(f"FAISS index built with {index.ntotal} vectors.")


FAISS index built with 94 vectors.


In [8]:
def search_faiss(query, k=5):
    # Embed the query
    query_embedding = model.encode(query, convert_to_numpy=True)
    faiss.normalize_L2(query_embedding.reshape(1, -1))

    # Search the index
    distances, indices = index.search(query_embedding.reshape(1, -1), k)

    # Fetch the top-k results from the dataset
    results = [chunks[i] for i in indices[0]]
    return results


In [9]:
results = search_faiss("أفضل الأماكن التاريخية في القاهرة")
for i, res in enumerate(results, 1):
    print(f"Result {i}:\n{res}\n{'-'*50}")


Result 1:
المتحف المصري الجديد
--------------------------------------------------
Result 2:
أهرامات الجيزة وتمثال أبو الهول
--------------------------------------------------
Result 3:
الموقع 
توجد القلعة في حيالخليفةبالمقطم، القاهرة
--------------------------------------------------
Result 4:
أهم الأنشطة في باب زويلة 
ربما تعتقد أن باب زويلة ليس إلا معلم سياحي لمشاهدته، لكن التجربة بأكملها في المنطقة التي يقع بها وكونها جزء من واحدة من أكبر الدول الحاكمة في مصر له تأثير خاص جعلته من أفضل اماكن سياحية في القاهرة يجب ترشيحها؛ وإليك اقتراحاتنا لأهم الأنشطة هناك
--------------------------------------------------
Result 5:
برج القاهرة
--------------------------------------------------


In [12]:
results = search_faiss("ما هي أشهر المعالم في حي الحسين؟")
for i, res in enumerate(results, 1):
    print(f"Result {i}:\n{res}\n{'-'*50}")


Result 1:
شارع المعز ومسجد الإمام الحسين
--------------------------------------------------
Result 2:
أهم الأنشطة في شارع المعز والحسين 
تعتبر منطقة الأزهر وشارع المعز والحسين من اجمل الاماكن في القاهرة التي تتسم بالحياة الليلة المزدحمة، حيث تضيء الفوانيس العتيقة الطريق وتعطيك جوًا ساحرًا يشجعك على ممارسة العديد من الأنشطة، مثل
حضور الندوات والحفلات في البيوت الثقافية؛ومن أهم البيوت الثقافية في الحسين بيت السناري، بيت السميحي، وكالة الغوري، قبة الغوري، بيت زينب خاتون
--------------------------------------------------
Result 3:
الموقع 
يوجد شارع المعز ومسجد الإمام الحسين في القاهرة القديمة، داخل منطقة العتبة عند شارع الأزهر، القاهرة أقرب محطة مترو العتبة
--------------------------------------------------
Result 4:
أهرامات الجيزة وتمثال أبو الهول
--------------------------------------------------
Result 5:
مميزات سوق خان الخليلي 
إذا كنت من محبي اماكن اثرية في القاهرة فلا ينبغي تفويت زيارة سوق خان الخليلي والاستمتاع بتجارب السياح هناك؛ ومن أبرز ما يوجد هناك
------------------------------

In [13]:
results = search_faiss("كيف تعكس المساجد والكنائس في القاهرة التنوع الديني؟")
for i, res in enumerate(results, 1):
    print(f"Result {i}:\n{res}\n{'-'*50}")

Result 1:
أهم الأنشطة 
ماذا تفعل في الكنيسة المُعلقة؟ إليك الإجابة
--------------------------------------------------
Result 2:
أوقات الزيارة 
يمكنك زيارة مسجد عمرو بن العاص في أي وقت
--------------------------------------------------
Result 3:
أهم الأنشطة 
 توفر القلعة المناظر البانورامية للقاهرة تجربة لا تُنسى
 مشاهدة واكتشاف الزخارف التي تتسم بها القلعة من الداخل
 مشاهدة المجسمات الموضوعة في ساحة القلعة
 أخذ الصور التذكارية مع مختلف الأروقة في القلعة وبجانب نوافذها المطلة على منطقة القاهرة التاريخية وجمع الأديان
--------------------------------------------------
Result 4:
أهم الأنشطة 
مسجد عمرو بن العاص ليس فقط موقعاً تاريخياً هاماً، بل هو أيضاً مركز نشط للفعاليات الدينية والثقافية إليك ما يمكنك القيام به هناك
--------------------------------------------------
Result 5:
الموقع 
يقع مسجد عمرو بن العاص في قلب مدينة القاهرة القديمة، عند ميدان جامع الكفور، وبالقرب من حصن بابل الروماني القديم ومنطقة القاهرة القبطية
ويمكنك الذهاب إلى المسجد بالمترو عبر النزول في محطة مارجرجس ثم مواصلة الم

In [15]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00


In [18]:
import gradio as gr

# === Gradio Wrapper Function ===
def gradio_search(query):
    results = search_faiss(query, k=5)
    output = ""
    for i, res in enumerate(results, 1):
        output += f"Result {i}:\n{res}\n{'-'*50}\n"
    return output

# === Launch Gradio Interface ===
iface = gr.Interface(
    fn=gradio_search,
    inputs=gr.Textbox(label="❓ اطرح سؤالك عن الأماكن السياحية في القاهرة"),
    outputs=gr.Textbox(label="🔎 النتائج", lines=20),
    title="Cairo Tourism Search (RAG-based)",
    description="أدخل سؤالك عن معالم القاهرة السياحية وسنسترجع أفضل النتائج من الدليل السياحي.",
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8ac915d879148579a6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
